CHAPTER 05 트리알고리즘  
05-3 트리의 앙상블

[앙상블 학습]  
: 정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘    
 > 대부분 결정 트리를 기반으로 만들어져 있음  

[랜덤 포레스트]  
: 앙상블 학습의 대표 주자(앙상블 학습을 적용할 때 가장 먼저 시도해 볼 것)   
: 결정 트리를 랜덤하게 만들어 결정 트리의 숲을 만들고, 각 결정 트리의 예측을 사용해 최종 예측을 만듦  
 
 <방법>  
- 각 트리를 훈련하기 위한 데이터를 랜덤하게 만듦  
   > 부트스트랩 샘플(데이터 세트에서 중복을 허용하여 데이터를 샘플링한 샘플)  
- 분류 모델인 RandomForestClassifier는 전체 특성 개수의 제곱근만큼의 특성을 선택  
   > 4개의 특성이 있다면 노드마다 2개를 랜덤하게 선택하여 사용  
- 회귀 모델인 RandomForestRegressor는 전체 특성을 사용  
- 사이킷런의 랜덤 포레스트는 기본적으로 100개의 결정 트리를 훈련  
- 분류일 때는 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼음  
- 회귀일 때는 각 트리의 예측을 평균

랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에  
훈련 세트에 과대적합되는 것을 막아주고 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있음  

In [5]:
# 데이터 불러오기 및 훈련 세트와 테스트 세트로 나누기
import numpy as np  
import pandas as pd  
from sklearn.model_selection import train_test_split  
wine = pd.read_csv('http://bit.ly/wine_csv_data')  
data = wine[['alcohol','sugar','pH']]  
target = wine['class']  
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [13]:
# cross_validate() 함수를 이용한 교차 검증 수행
from sklearn.model_selection import cross_validate  
from sklearn.ensemble import RandomForestClassifier  
rf = RandomForestClassifier(n_jobs = -1, random_state=42)
scores = cross_validate(rf, train_input, train_target, 
                       return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


- 랜덤포레스트는 기본적으로 100개의 결정 트리를 이용하므로 n_jobs 매개변수를 -1로 지정해 모든 CPU 코어를 사용하는 것이 좋음  
- return_train_score=True >  검증 점수, 훈련 세트 점수 같이 반환(기본값 False)

In [ ]:
rf.fit(train_input, train_target) # 랜덤 포레스트 모델 훈련
print(rf.feature_importances_) # 특성 중요도 출력

[0.23167441 0.50039841 0.26792718]


랜덤 포레스트는 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기때문에  
하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻음  
> 과대적합을 줄이고 일반화 성능을 높이는데 도움  

[OOB(out of bag) 샘플]  
: 훈련 세트에서 중복을 허용하여 부트스트램 샘플을 만들 때, 부트스트램 샘플에 포함되지 않고 남는 샘플  
- 이 샘플을 이용하여 부트스트랩 샘플로 훈련한 결정 트리를 평가할 수 있음(검증 세트의 역할)   
- 'oob_score=True'로 지정하면 각 결정 트리의 OOB 점수를 평군하여 출력함  
- OOB 점수를 사용하면 교차 검증을 대신 할 수 있어 훈련 세트에 더 많은 샘플을 사용할 수 있음

In [16]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)  
rf.fit(train_input, train_target)  
print(rf.oob_score_)

0.8934000384837406


[엑스트라 트리]  
: 랜덤 포레스트와 비슷하나 부트스트램 샘플을 사용하지 않음  
- 각 결정 트리를 만들 때 전체 훈련 세트를 사용  
- 대신 노드를 분할 할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할  
- splitter='random'인 결정 트리
- 무작위성이 더 크기 때문에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야하지만, 랜덤하게 노드를 분할하기 때문에 속도가 빠름

In [17]:
from sklearn.ensemble import ExtraTreesClassifier  
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)  
scores = cross_validate(et, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_) # 특성 중요도

[0.20183568 0.52242907 0.27573525]


[그레이디언트 부스팅]  
: 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블하는 방법  
- 기본적으로 깊이가 3인 결정 트리를 100개 사용함  
- 깊이가 얕은 결정 트리를 사용하기 때문에 과대적합에 강하고 높은 일반화 성능 기대할 수 있음  
- 경사하강법을 사용하여 트리를 앙상블에 추가(분류는 로지스틱 손실 함수, 회귀는 평균 제곱 오차 함수 사용)

In [22]:
# GradientBoostingClassifier를 사용해 데이터셋의 교차 검증 점수 확인
from sklearn.ensemble import GradientBoostingClassifier  
gb = GradientBoostingClassifier(random_state=42)  
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


그레이디언트 부스팅은 결정 트리의 개수를 늘려도 과대적합에 강한 것을 확인

In [ ]:
# 성능 향상을 위해 학습률 증가, 트리의 개수 늘려 학습
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42) 
score = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


n_estimators=500 : 결정 트리 개수 500개로 5배 늘림  
learning_rate=0.2 : 학습률 2배로 올림  

In [26]:
gb.fit(train_input, train_target)
print(gb.feature_importances_) # 특성 중요도 확인

[0.15887763 0.6799705  0.16115187]


- 일반적으로 그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 얻을 수 있음  
- 하지만 순서대로 트리를 추가하기 때문에 훈련 속도 느림  
- GradientBoostingClassifier에는 n_jobs 매개변수가 없음  
- 그레이디언트 부스팅의 회귀버전은 GradientBoostingRegressor  

[히스토그램 기반 그레이디언트 부스팅]  
- 정형 데이터 머신러닝 알고리즘 중 가장 인기가 높은 알고리즘  
- 입력 특성을 256개의 구간으로 나눔   
  : 노드를 분할할 때 최적의 분할을 빠르게 찾을 수 있음  
  > 트리가 분할 기준을 찾을 때 모든 실숫값을 볼 필요 없이 각 구간 단위로만 계산하기 때문    

In [27]:
# HistGradientBoostingClassifier  
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)  
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


히스토그램 기반 그레이디언트 부스팅은 자체적으로 특성 중요도를 제공하지 않음    
>  permutation_importance() 함수 사용  
   : 특성을 하나씩 랜덤하게 섞어 모델의 성능 변화를 관찰해 어떤 특성이 중요한지 계산

In [ ]:
# permutation_importance() 함수를 사용해 훈련 세트 특성 중요도 계산
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [30]:
# 테스트 세트 특성 중요도 계산
result = permutation_importance(hgb, test_input, test_target,
                                n_repeats=10, random_state=42, n_jobs=-1)  # 특성 중요도, 평균, 표준편차 반환
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [31]:
# 테스트 세트에서의 성능 확인
hgb.score(test_input, test_target)

0.8723076923076923

In [ ]:
!pip install xgboost # xgboost 설치

In [36]:
# XGBoost 라이브러리의 히스토그램 기반 그레이언트 부스팅 알고리즘
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target,   # 사이킷런의 cross_validate()함수와 함께 사용 가능
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9567059184812372 0.8783915747390243


In [38]:
!pip install lightgbm # lightgbm 설치

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 3.8 MB/s  0:00:00 eta 0:00:01


In [40]:
# LightGBM 라이브러리의 히스토그램 기반 그레이디언트 부스팅
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, 
                        return_train_score=True, n_jobs=-1)
print(np.mean(score['train_score']), np.mean(score['test_score']))

[LightGBM] [Info] Number of positive: 3151, number of negative: 1007
[LightGBM] [Info] Number of positive: 3151, number of negative: 1007
[LightGBM] [Info] Number of positive: 3152, number of negative: 1006
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.028998 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 373
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.031181 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 371
[LightGBM] [Info] Number of data points in the train set: 4158, number of used features: 3
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.040866 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 372
[LightGBM] [Info] Number of data points in the train 

[정리] 

1. 랜덤 포레스트    
- 가장 대표적인 앙상블 학습 알고리즘  
- 성능이 좋고 안정적이기 때문에 첫번째로 시도해 볼 수 있는 앙상블 학습 중 하나  
- 결정 트리를 훈련하기 위해 부트스트랩 샘플을 만듦
- 전체 특성 중 일부를 랜덤하게 선택하여 결정 트리 만듦

2. 엑스트라 트리  
- 랜덤 포레스트와 비슷하지만 부트스트랩 샘플을 사용하지 않음  
- 노드를 분할할 때 랜덤하게 분할  
- 랜덤 포레스트보다 훈련 속도가 빠르지만 보통 더 많은 트리가 필요  

3. 그레이디언트 부스팅  
- 깊이가 얕은 트리를 연속적으로 추가하여 손실 함수를 최소화하는 앙상블 방식  
- 성능이 뛰어나지만 병렬로 훈련할 수 없기 때문에 랜덤,엑스트라 보다 속도 느림  
- 학습률 매개변수를 조정해 모델의 복잡도 제어 가능  
- 학습률 매개변수가 크면 복잡하고 훈련세트에 과대적합된 모델을 얻을 수 있음  

4. 히스토그램 기반 그레이디언트 부스팅  
- 가장 뛰어난 앙상블 학습으로 평가 받음  
- 훈련 데이터를 256개 구간으로 변환하여 사용하기 때문에 노드 분할 속도가 매우 빠름  
